In [ ]:
import os, re, json, math
from collections import Counter, defaultdict

DATA_PATH_DEFAULT = os.environ.get("CAFE_CHUNKS_PATH", "/content/cafe_chunks.jsonl")
TOPK_DEFAULT = 5

# ---------- 문자열 전처리 ----------
def normalize(s: str) -> str:
    if s is None: return ""
    s = s.strip()
    #s = re.sub(r"[^0-9A-Za-z\u3131-\u318E\uAC00-\uD7A3\s\.\,\?\!\-\~·]+", " ", s)
    #s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^0-9A-Za-z\u3131-\u318E\uAC00-\uD7A3\s]+", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()

def bm25_tokenize(s: str):
    s = normalize(s).lower()
    return s.split() if s else []

def split_chunk_to_tokens(ch: str):
    ch = normalize(ch)
    return ch.split() if ch else []

def split_list_chunks_to_tokens(chunks):
    toks = []
    for ch in chunks or []:
        toks.extend(split_chunk_to_tokens(str(ch)))
    return toks

# ---------- 데이터 로딩 ----------
def load_step_examples(path: str):
    examples = []
    if not os.path.exists(path):
        return examples
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if not line: continue
            try:
                ex=json.loads(line)
                if "selected_chunks" in ex and "target_chunk" in ex:
                    examples.append(ex)
            except Exception:
                pass
    return examples

# ---------- n-gram 유사도 (STT fuzzy match용) ----------
def char_ngrams(s: str, n=3):
    s = normalize(s)
    s = re.sub(r"\s+", " ", s)
    if len(s) < n:
        return {s} if s else set()
    return {s[i:i+n] for i in range(len(s)-n+1)}

def jaccard(a: set, b: set):
    if not a and not b: return 1.0
    if not a or not b: return 0.0
    inter = len(a & b)
    union = len(a | b)
    return inter / union if union else 0.0

# ---------- 통계 만들기 ----------
def build_stats_from_examples(examples):
    vocab = Counter()
    trans = defaultdict(Counter)      # prev -> next
    trans2 = defaultdict(Counter)     # (prev2, prev1) -> next
    candidates = set()
    # STT별 시작 분포: user_asr_norm -> Counter(next_token)
    start_counts_by_asr = defaultdict(Counter)

    # 모든 user_asr 모음 (유사도 탐색용)
    asr_set = set()

    for ex in examples:
        user_asr = ex.get("user_asr","")
        user_asr_norm = normalize(user_asr)
        asr_set.add(user_asr_norm)

        selected = split_list_chunks_to_tokens(ex.get("selected_chunks", []))
        target_tokens = split_chunk_to_tokens(ex.get("target_chunk",""))

        # 전역 vocab/후보는 target 기준
        for t in target_tokens:
            vocab[t] += 1
            candidates.add(t)

        # 시작 분포: selected가 빈 경우, target의 첫 토큰을 시작으로 간주
        if not selected and target_tokens:
            start_counts_by_asr[user_asr_norm][target_tokens[0]] += 1

        # 전이: selected 뒤에 target이 순차적으로 이어진다고 가정
        ctx = list(selected)
        for tok in target_tokens:
            if len(ctx) >= 1:
                trans[ctx[-1]][tok] += 1
            if len(ctx) >= 2:
                trans2[(ctx[-2], ctx[-1])][tok] += 1
            ctx.append(tok)

    return vocab, trans, trans2, candidates, start_counts_by_asr, asr_set

# ---------- 점수 계산 ----------
def transition_score(candidate, trans, trans2, ctx_tail):
    eps = 1e-9
    s = 0.0
    if len(ctx_tail) >= 2:
        prev2 = (ctx_tail[-2], ctx_tail[-1])
        tbl2 = trans2.get(prev2, {})
        tot2 = sum(tbl2.values())
        if tot2 > 0:
            s += 0.7 * ((tbl2.get(candidate,0)+eps)/(tot2+eps))
    if len(ctx_tail) >= 1:
        prev1 = ctx_tail[-1]
        tbl1 = trans.get(prev1, {})
        tot1 = sum(tbl1.values())
        if tot1 > 0:
            s += 0.3 * ((tbl1.get(candidate,0)+eps)/(tot1+eps))
    return s

def freq_prior_score(candidate, vocab: Counter):
    tot = sum(vocab.values()) or 1.0
    return vocab.get(candidate, 0) / tot

def lexical_match_score(candidate, user_asr, ctx_tail):
    q = set(bm25_tokenize(user_asr))
    c = set(bm25_tokenize(candidate))
    if not c: return 0.0
    overlap = len(q & c)
    bonus = 0.0
    if ctx_tail:
        last = ctx_tail[-1]
        if last and candidate and candidate[0] == last[0]:
            bonus += 0.03
    return overlap + bonus

# ---------- 후보 수집 ----------
def gather_candidates(candidates, trans, trans2, context_tokens, vocab, start_prior=None, widen=True):
    cand = set()
    # 전이 기반
    if context_tokens:
        cand |= set(trans.get(context_tokens[-1], {}).keys())
    if len(context_tokens) >= 2:
        cand |= set(trans2.get((context_tokens[-2], context_tokens[-1]), {}).keys())
    # 시작 분포 기반(컨텍스트가 없고 start_prior가 있으면 우선 추가)
    if not context_tokens and start_prior:
        cand |= set(start_prior.keys())
    # widen
    if widen or not cand:
        cand |= {w for w, _ in vocab.most_common(300)}
    cand |= set(candidates)
    return [c for c in cand if c]

# ---------- STT 조건부 시작 점수(보간) ----------
def asr_start_score(candidate, user_asr_input, start_counts_by_asr, asr_set):
    """입력 STT와 가장 유사한 STT의 시작 분포를 찾아 점수로 반환."""
    user_norm = normalize(user_asr_input)
    cand_score = 0.0

    # 1) 정확히 같은 STT가 있으면 그 분포 사용
    if user_norm in start_counts_by_asr:
        sc = start_counts_by_asr[user_norm]
        tot = sum(sc.values()) or 1.0
        cand_score = sc.get(candidate, 0) / tot

    # 2) 유사 STT fallback (3-gram Jaccard)
    if cand_score == 0.0 and asr_set:
        a_ngrams = char_ngrams(user_norm, n=3)
        best_sim, best_asr = 0.0, None
        for a in asr_set:
            sim = jaccard(a_ngrams, char_ngrams(a, n=3))
            if sim > best_sim:
                best_sim, best_asr = sim, a
        if best_asr is not None and best_sim >= 0.20:  # 임계값 조정
            sc = start_counts_by_asr.get(best_asr, {})
            tot = sum(sc.values()) or 1.0
            cand_score = sc.get(candidate, 0) / tot
        # 너무 낮으면 사실상 0으로 둔다

    return cand_score

# ---------- 랭킹 ----------
def rank_candidates(cand_pool, user_asr, context_tokens, vocab, trans, trans2,
                    start_counts_by_asr, asr_set, k=5):
    counts_in_ctx = Counter(context_tokens)
    out = []
    for c in cand_pool:
        if not c: continue

        # 보간 가중치: 시작이면 STT 시작분포 비중을 크게
        if not context_tokens:
            # 시작 상태일 때: asr_start 0.6 + prior 0.25 + lexical 0.05 + tiny transition(0.1)
            s_asr = asr_start_score(c, user_asr, start_counts_by_asr, asr_set)
            s = (0.60 * s_asr
                 + 0.10 * transition_score(c, trans, trans2, context_tokens)
                 + 0.05 * lexical_match_score(c, user_asr, context_tokens)
                 + 0.25 * freq_prior_score(c, vocab))
        else:
            # 진행 중: transition 0.65 + prior 0.25 + lexical 0.10
            s = (0.65 * transition_score(c, trans, trans2, context_tokens)
                 + 0.10 * lexical_match_score(c, user_asr, context_tokens)
                 + 0.25 * freq_prior_score(c, vocab))

        # 반복 패널티
        s *= (1.0 / (1.0 + counts_in_ctx.get(c, 0)))
        out.append((c, s))

    out.sort(key=lambda x: (-x[1], len(x[0])))
    dedup, seen = [], set()
    for c, s in out:
        if c in seen: continue
        seen.add(c); dedup.append((c, s))
        if len(dedup) >= k: break
    return dedup

# ---------- 메인 ----------
class StepwiseChunkPicker:
    def __init__(self, data_path: str = DATA_PATH_DEFAULT):
        self.examples = load_step_examples(data_path)
        if not self.examples:
            raise RuntimeError(f"데이터가 비었습니다: {data_path}")
        (self.vocab, self.trans, self.trans2, self.candidates,
         self.start_counts_by_asr, self.asr_set) = build_stats_from_examples(self.examples)

    def suggest(self, user_asr: str, selected_chunks_tokens, topk=TOPK_DEFAULT):
        start_prior = None
        if not selected_chunks_tokens:
            # 해당 STT(또는 근접 STT)의 시작 분포를 cand pool 생성에 반영
            # cand pool 내부에서 가중치는 rank_candidates에서 다시 적용
            user_norm = normalize(user_asr)
            start_prior = self.start_counts_by_asr.get(user_norm, None)
            if start_prior is None:
                # 근접 STT 하나 골라서 start_prior로 사용
                best_asr = None
                best_sim = 0.0
                a_ngrams = char_ngrams(user_norm, n=3)
                for a in self.asr_set:
                    sim = jaccard(a_ngrams, char_ngrams(a, n=3))
                    if sim > best_sim:
                        best_sim, best_asr = sim, a
                if best_asr and best_sim >= 0.25:
                    start_prior = self.start_counts_by_asr.get(best_asr, None)

        cand_pool = gather_candidates(self.candidates, self.trans, self.trans2,
                                      selected_chunks_tokens, self.vocab,
                                      start_prior=start_prior, widen=True)
        ranked = rank_candidates(cand_pool, user_asr, selected_chunks_tokens,
                                 self.vocab, self.trans, self.trans2,
                                 self.start_counts_by_asr, self.asr_set, k=topk)
        return ranked

    def simulate_app_input(self, topk=TOPK_DEFAULT):
        user_asr = input("👂 점원(STT) 말 입력: ").strip()
        context_tokens = []
        steps = 0
        while True:
            ranked = self.suggest(user_asr, context_tokens, topk=topk)
            print("\n👉 후보 청크:")
            for i, (c, s) in enumerate(ranked, 1):
                print(f"{i}. {c} (score={s:.3f})")
            sel = input("선택 번호 (엔터=완성): ").strip()
            if sel == "": break
            try:
                idx = int(sel)
                if not (1 <= idx <= len(ranked)):
                    print("⚠️ 올바른 번호를 입력하세요."); continue
            except Exception:
                print("⚠️ 숫자를 입력하세요."); continue
            chosen = ranked[idx-1][0]
            context_tokens.append(chosen)
            print(f"✅ 선택됨: {chosen}")
            print("🧱 현재까지:", " ".join(context_tokens))
            steps += 1
            if steps >= 50:
                print("ℹ️ 최대 50개 청크까지 입력 가능합니다. 자동 완료합니다.")
                break
        print("\n🗣️ 최종 문장:", " ".join(context_tokens))
        return " ".join(context_tokens)

# -------- 편의 함수 --------
def simulate_app_input():
    picker = StepwiseChunkPicker(DATA_PATH_DEFAULT)
    return picker.simulate_app_input(topk=TOPK_DEFAULT)

def suggest_next(user_asr: str, selected_chunks):
    picker = StepwiseChunkPicker(DATA_PATH_DEFAULT)
    ctx = split_list_chunks_to_tokens(selected_chunks)
    return picker.suggest(user_asr, ctx, topk=TOPK_DEFAULT)

print("✅ 준비 완료!  data:", DATA_PATH_DEFAULT)
print("예) simulate_app_input()  또는  suggest_next('주문 도와드릴게요', ['아이스','모카'])")
